In [1]:
import pandas as pd
import nltk
from tqdm import tqdm
from nltk import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()
#from Levenshtein import distance
import psycopg2
import random
import json
from string import punctuation


# eng

In [2]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
    
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# TUR

In [7]:
from TurkishStemmer import TurkishStemmer
stemmer = TurkishStemmer()
import requests

['мама_NOUN_мама_10_1_1', 'мыла_NOUN_мыло_10_1_1']

In [8]:
def get_word_pos_lemma_id_list_tur(clean_ngramm,word_id, ref_id, set_id):
    headers_tr = {
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'postman-token': 'c18af364-c1cb-cc41-0903-063547ac7fce',
    }
    url_tr = 'http://localhost:8081/analyze'
    
    final_ngramm = []
    tkn = nltk.word_tokenize(clean_ngramm)
    #print(tkn)
    for word in tkn:
        data = {"tokens" : [word]}
        data_dump = json.dumps(data)
        r = requests.post(url = url_tr, data=data_dump, headers=headers_tr)
        lemma = stemmer.stem(word)
        pos = r.text.split("+")[1].lower()
        element = word + "_" + pos + "_" + lemma + '_' + str(word_id) + '_' + str(ref_id) + '_' + str(set_id)
        final_ngramm.append(element)
    return final_ngramm
get_word_pos_lemma_id_list_tur("merhaba", 1,1,1,)

['merhaba_noun_merhap_1_1_1']

# RUS

In [ ]:
def get_word_pos_lemma_id_list_rus(clean_ngramm,word_id, ref_id, set_id):
    final_ngramm = []
    tkn = nltk.word_tokenize(clean_ngramm)
    #print(tkn)
    for word in tkn:
        p = morph.parse(word)[0]
        lemma = p.normal_form
        pos = p.tag.POS
        #print(lemma,pos)
        if not pos:
            pos = "nodetected"
        element = word + "_" + pos + "_" + lemma + '_' + str(word_id) + '_' + str(ref_id) + '_' + str(set_id)
        final_ngramm.append(element)
    return final_ngramm
get_word_pos_lemma_id_list("мама мыла", 10,1,1)

In [13]:
def  get_lemmatized_popular_ngramm(popular_ngramm, popular_ngramm_ids,popular_ngramm_ref_ids,popular_ngramm_set_ids, debug = False):
    ngramm_list = []
    for word_ind in tqdm(range(len(popular_ngramm))):
        word_id = popular_ngramm_ids[word_ind]
        ngramm = popular_ngramm[word_ind]
        ref_id = popular_ngramm_ref_ids[word_ind]
        set_id = popular_ngramm_set_ids[word_ind]
        clean_ngramm = ''
        for char in ngramm:
            if char not in punctuation: #punctuation_small_set
                clean_ngramm += char
            else:
                clean_ngramm += ' '
        if debug: print(clean_ngramm)
        final_ngramm = get_word_pos_lemma_id_list_tur(clean_ngramm,word_id,ref_id,set_id)
        ngramm_list.append(final_ngramm)

    return ngramm_list

In [15]:
def get_alph_dict(ngramms_pos_lemm_list):
    alph_dict = {}
    for ngramm in ngramms_pos_lemm_list:
        for elemetn in ngramm:
            word = elemetn.split("_")[0]
            first_two_el = min(2,len(word))
            first_two_letters = word[:first_two_el]
            if first_two_letters in alph_dict:
                alph_dict[first_two_letters].append(ngramm)
            else:
                alph_dict[first_two_letters] = []
                alph_dict[first_two_letters].append(ngramm)
    return alph_dict

In [10]:
def get_top_prop(db):
    popular_qgramms = pd.read_csv(db)
    popular_qgramms_list = list(popular_qgramms['translate'])
    ppopular_qgramms_list_id = list(popular_qgramms['word_id'])
    popular_qgramms_ref_id = list(popular_qgramms['ref_id'])
    popular_qgramms_set_id = list(popular_qgramms['setting_id'])
    print(popular_qgramms.head())
    return popular_qgramms_list, ppopular_qgramms_list_id, popular_qgramms_ref_id, popular_qgramms_set_id

In [20]:
def save_reference(prefix, word_list, id_list, ref_id_list, set_id_list):
    ngramm = get_lemmatized_popular_ngramm(word_list,id_list,ref_id_list,set_id_list)
    ngramm_dict = get_alph_dict(ngramm)
    with open(prefix +"gramms_top.json" ,"w") as f:
        json.dump(ngramm, f, indent = 4, ensure_ascii=False)
    with open(prefix + "gramms_alpha_dict.json" ,"w") as f:
        json.dump(ngramm_dict, f, indent = 4, ensure_ascii=False)

In [18]:
def get_db_return_ref_files(prefix, db):
    word_list, id_list, ref_id_list, set_id_list= get_top_prop(db)
    save_reference(prefix, word_list, id_list, ref_id_list, set_id_list)
    

In [19]:
get_db_return_ref_files("bi", "top_10k_bigram.csv")

  0%|          | 0/10000 [00:00<?, ?it/s]

   word_id  ref_id  setting_id          translate
0    85345       2           1  dağıtmak, serpmek
1   235754       2           1       Frenk soğanı
2    17211       2           1     ateş püskürmek
3    35736       2           1    eğitimle ilgili
4   188514       2           1       resim yazısı


100%|██████████| 10000/10000 [09:42<00:00, 19.17it/s]


In [21]:
get_db_return_ref_files("tri", "top_10k_trigram.csv")

  0%|          | 0/5444 [00:00<?, ?it/s]

   word_id  ref_id  setting_id                   translate
0    23447       2           1  başlatmak, piyasaya sürmek
1    36537       2           1        hizmet etmek, sunmak
2     1908       2           1       davranmak, rol yapmak
3  1042433       2           1   yasa tasarısını reddetmek
4    23522       2           1         götürmek, yol açmak


100%|██████████| 5444/5444 [09:00<00:00, 10.07it/s]


In [23]:
get_db_return_ref_files("q", "top_10k_qgram.csv")

  0%|          | 1/2355 [00:00<04:56,  7.95it/s]

   word_id  ref_id  setting_id                                translate
0    23715       2           1              izin vermek, olanak tanımak
1  6190377       2           1                  Her şey yoluna girecek.
2  6260909       2           1              Henüz bu konuyu düşünmedim.
3  6191274       2           1  Doğru yoldasınız. Doğru düşünüyorsunuz.
4     1641       2           1                (gemi, uçak, tren) içinde


100%|██████████| 2355/2355 [04:33<00:00,  8.60it/s]
